# Melanoma Classification

In [2]:
import cv2
import os
import numpy as np
from keras.layers import Activation
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Input
from keras.layers import merge
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.convolutional import ZeroPadding2D
from keras.models import Model
from keras.models import Sequential
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from skimage.transform import rescale, resize
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
import tensorflow as tf

c:\users\huzai\appdata\local\programs\python\python35\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
def disp(I):
    I = np.uint8(I)
    cv2.imshow('image', I)
    cv2.waitKey(0)

In [4]:
datagen = ImageDataGenerator(rescale = 1.0 / 255)

In [5]:
gt_path = 'Resized_Data/Ground_truth/gt'
melanoma_input = 'Resized_Data/Melanoma/melanoma'
melanoma_output_training = 'Partitioned_Data/Training/1_Melanoma'
melanoma_output_validation = 'Partitioned_Data/Validation/1_Melanoma'
others_input = 'Resized_Data/Others/others'
others_output_training = 'Partitioned_Data/Training/0_Others'
others_output_validation = 'Partitioned_Data/Validation/0_Others'

### Image Resize and Partitioning

Image is multiplied with the ground truth to give the region of interest and then resized the input size of the CNN ( 64 X 64 ).
It is partitioned 90:10 ratio between training and testing

In [6]:
counter = 0
melanoma_length = len(os.listdir(melanoma_input))
for image_name in os.listdir(melanoma_input):
    counter = counter + 1
    image = cv2.imread(os.path.join(melanoma_input,image_name))
    ground_image_name = image_name[:12] + '_segmentation.png'
    gt_image = cv2.imread(os.path.join(gt_path,ground_image_name))
    gt_image = cv2.threshold(gt_image,127,255,cv2.THRESH_BINARY)[1]
    gt_image = gt_image / 255.0
    image = image * gt_image
    image = cv2.resize(image, (64,64))
    if counter <= 0.9 * melanoma_length:
        cv2.imwrite(os.path.join(melanoma_output_training, image_name), image)
    else:
        cv2.imwrite(os.path.join(melanoma_output_validation, image_name), image)
    
counter = 0
others_length = len(os.listdir(others_input))
for image_name in os.listdir(others_input):
    counter = counter + 1
    image = cv2.imread(os.path.join(others_input,image_name))
    ground_image_name = image_name[:12] + '_segmentation.png'
    gt_image = cv2.imread(os.path.join(gt_path,ground_image_name))
    gt_image = cv2.threshold(gt_image,127,255,cv2.THRESH_BINARY)[1]
    gt_image = gt_image / 255.0
    image = image * gt_image
    image = cv2.resize(image, (64,64))
    if counter <= 0.9 * others_length:
        cv2.imwrite(os.path.join(others_output_training, image_name), image)
    else:
        cv2.imwrite(os.path.join(others_output_validation, image_name), image)

### CNN

In [7]:
CNN_input_size = [64, 64, 3]
trained_classifier_path = 'Trained/classifier.h5'

### F1-loss
This loss was designed to directly maximize the f1-score.  
Using this loss did not give much improvement.

In [8]:
def f1_loss(y_true, y_pred):
    TP = K.sum(y_true * y_pred)
    soft_precision_positive = TP / (K.sum(y_pred) + 1e-7)
    soft_recall_positive = TP / (K.sum(y_true) + 1e-7)
    f1_loss_positive = 2 * soft_precision_positive * soft_recall_positive / (soft_precision_positive + soft_recall_positive + 1e-7)
    TN = K.sum((1 - y_true) * (1 - y_pred))
    soft_precision_negative = TN / (K.sum(1 - y_pred) + 1e-7)
    soft_recall_negative = TN / (K.sum(1 - y_true) + 1e-7)
    f1_loss_negative = 2 * soft_precision_negative * soft_recall_negative / (soft_precision_negative + soft_recall_negative + 1e-7)
    return 1 - (0.99 * f1_loss_positive + 0.01 * f1_loss_negative)

### Weighted Binary Cross-Entropy
This loss was designed to balance the skewness of the dataset towards the non-melanoma images.  
This loss had been used but did not give much improvement.

In [9]:
def weighted_binary_crossentropy(y_true, y_pred):
    return -K.sum(0.99 * y_true * K.log(y_pred + 1e-7) + 0.01 * (1 - y_true) * K.log(1 - y_pred + 1e-7))

### Architecture

In [14]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape = CNN_input_size, activation = 'relu', padding = 'same', name = 'conv1'))
model.add(MaxPooling2D((2, 2), strides = (2, 2)))
model.add(Conv2D(32, (3, 3), input_shape = CNN_input_size, activation = 'relu', padding = 'same', name = 'conv2'))
model.add(MaxPooling2D((2, 2), strides = (2, 2)))
model.add(Conv2D(32, (3, 3), input_shape = CNN_input_size, activation = 'relu', padding = 'same', name = 'conv3'))
model.add(MaxPooling2D((2, 2), strides = (2, 2)))
model.add(Conv2D(32, (3, 3), input_shape = CNN_input_size, activation = 'relu', padding = 'same', name = 'conv4'))
model.add(MaxPooling2D((2, 2), strides = (2, 2)))
model.add(Flatten())

model.add(Dense(128, activation = 'relu', name = 'dense_1'))
model.add(Dense(1, name = 'dense_2'))
model.add(Dropout(0.5))
model.add(Activation('sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               (None, 64, 64, 32)        896       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
conv2 (Conv2D)               (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
conv3 (Conv2D)               (None, 16, 16, 32)        9248      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 8, 8, 32)          0         
_________________________________________________________________
conv4 (Conv2D)               (None, 8, 8, 32)          9248      
__________

### Training

In [15]:
callbacks = [ModelCheckpoint(trained_classifier_path, monitor = 'val_loss', save_best_only = True, verbose = 2)]

In [16]:
model.fit_generator(datagen.flow_from_directory('Partitioned_Data/Training', batch_size = 16, 
                    class_mode = 'binary', target_size = (64, 64)), steps_per_epoch = 1800 / 4, epochs = 18, 
                    verbose = 1, validation_data = datagen.flow_from_directory('Partitioned_Data/Validation', 
                    batch_size = 4, class_mode = 'binary',  target_size = (64, 64)), callbacks = callbacks, class_weight = {0:1, 1:8})

Found 1799 images belonging to 2 classes.
Found 201 images belonging to 2 classes.
Epoch 1/18
450/450 [==============================] - 10s 23ms/step - loss: 1.5301 - acc: 0.5591 - val_loss: 0.7828 - val_acc: 0.2438

Epoch 00001: val_loss improved from inf to 0.78284, saving model to Trained/classifier.h5
Epoch 2/18
450/450 [==============================] - 10s 22ms/step - loss: 1.4848 - acc: 0.6281 - val_loss: 0.8286 - val_acc: 0.2786

Epoch 00002: val_loss did not improve
Epoch 3/18
450/450 [==============================] - 10s 21ms/step - loss: 1.4441 - acc: 0.6564 - val_loss: 0.8038 - val_acc: 0.2537

Epoch 00003: val_loss did not improve
Epoch 4/18
450/450 [==============================] - 10s 21ms/step - loss: 1.3670 - acc: 0.7134 - val_loss: 0.7792 - val_acc: 0.3831

Epoch 00004: val_loss improved from 0.78284 to 0.77920, saving model to Trained/classifier.h5
Epoch 5/18
450/450 [==============================] - 9s 21ms/step - loss: 1.2981 - acc: 0.7481 - val_loss: 0.8915 - 

### Testing

In [17]:
test_set = datagen.flow_from_directory('Partitioned_Data/Validation', target_size = (64, 64), class_mode = 'binary')

Found 201 images belonging to 2 classes.


In [18]:
results = model.predict_generator(test_set)
y_true = test_set.classes
y_pred = np.rint(results)
cm = confusion_matrix(y_true,y_pred)
print(cm)

[[127  36]
 [ 27  11]]


In [19]:
# Class - Melanoma
precision = cm[1][1] / (cm[1][1] + cm[0][1])
recall =  cm[1][1] / (cm[1][1] + cm[1][0])

f1 = 2 * precision * recall / (precision + recall)
print("F1 score for melanoma:", f1)

# Class - Others
precision = cm[0][0] / (cm[0][0] + cm[0][1])
recall = cm[0][0] / (cm[0][0] + cm[1][0])

f1 = 2 * precision * recall / (precision + recall)
print("F1 score for non-melanoma:", f1)

F1 score for melanoma: 0.2588235294117647
F1 score for non-melanoma: 0.8012618296529967


### Melonoma f1-score : 0.258
### Other f1-score : 0.801